In [1]:
import os

SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'

os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]")\
                            .appName("SparkStreamingKafkaBasic").getOrCreate()

22/04/01 21:22:11 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/04/01 21:22:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-625fca4b-32b4-43ab-a3ec-9e559e759670;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 724ms :: artifacts dl 11ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [4]:
kafkaDf = spark.readStream.format("kafka")\
              .option("kafka.bootstrap.servers", "localhost:9092")\
              .option("subscribe", "statewise-earning")\
              .load()

In [5]:
kafkaDf.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
stateDf = kafkaDf.selectExpr("timestamp", "CAST(value AS STRING)")
stateDf.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: string (nullable = true)



In [11]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType, DateType, LongType

schema = StructType([
            StructField("state", StringType(), True),
            StructField("amount", StringType(), True),
                        ]
)

In [12]:
jsonDf = stateDf.withColumn("value", F.from_json("value", schema))

In [15]:
stateDf1 = jsonDf.select("timestamp", F.col("value.*"))  

In [10]:
# echoOnconsole = stateDf1\
#                 .writeStream\
#                 .outputMode("append")\
#                 .format("console")\
#                 .start()

In [14]:
echoOnconsole.awaitTermination()

NameError: name 'echoOnconsole' is not defined